In [1]:
from pyspark.sql import SparkSession, functions as F, Row
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.functions import vector_to_array
from operator import add
import numpy as np
from math import log
import logging, os
import pandas as pd
from ucimlrepo import fetch_ucirepo
import math
from pyspark.ml.linalg import Vectors
import heapq

In [2]:
spark = (
    SparkSession.builder.appName("Dimension Reduction")
    .master("local[*]")
    .getOrCreate()
)


25/07/06 14:14:36 WARN Utils: Your hostname, Soroush resolves to a loopback address: 127.0.1.1; using 192.168.100.10 instead (on interface wlp5s0)
25/07/06 14:14:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/06 14:14:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Import Musk version 2 dataset

In [3]:

# fetch dataset
musk_version_2 = fetch_ucirepo(id=75)

# data (as pandas dataframes)
X = musk_version_2.data.features
y = musk_version_2.data.targets

pdf = pd.concat([X, y], axis=1)
df = spark.createDataFrame(pdf)

# grab column names
label_col = y.columns[0] if hasattr(y, "columns") else "class"
feature_cols = [c for c in df.columns if c != label_col]

#### Scale With Max_Min Normalization method

In [4]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")
assembled_df = assembler.transform(df)

# apply spark built-in min-max scaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(assembled_df)
df_scaled = scaler_model.transform(assembled_df)



25/07/06 14:17:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### Density Based Representation

In [5]:
# number of bins per feature
p = 10

cube_counts = (
    df_scaled.select("scaledFeatures").rdd
    # MAP: vector → tuple of bin indices
    .map(lambda row: tuple(int(min(x * p, p - 1)) for x in row.scaledFeatures))
    # MAP: tuple → (cube_id, 1)
    .map(lambda bins: ("_".join(map(str, bins)), 1))
    # REDUCE: sum counts per cube_id
    .reduceByKey(lambda a, b: a + b)
)



In [6]:
num_feats = len(feature_cols)

density_df = spark.createDataFrame(
    cube_counts.map(lambda kv: Row(cube_id=kv[0], density=kv[1]))
)

for i in range(num_feats):
    density_df = density_df.withColumn(
        f"g{i}", F.split(F.col("cube_id"), "_")[i].cast("int")
    )


#### mRMD-Based Relevant Subspace Selection

In [7]:
# RDD format
col_names = [f"g{i}" for i in range(num_feats)]
mr_rdd    = density_df.select(col_names + ["density"]).rdd.cache()
N_total   = mr_rdd.count()

# calculate similarity
fd_counts = (
    mr_rdd.flatMap(
        lambda row: [((j, getattr(row, col_names[j]), row.density), 1) for j in range(num_feats)]
    ).reduceByKey(lambda a, b: a + b)
)

feat_marg = fd_counts.map(lambda kv: ((kv[0][0], kv[0][1]), kv[1])).reduceByKey(lambda a, b: a + b)

dens_marg = fd_counts.map(lambda kv: (kv[0][2], kv[1])).reduceByKey(lambda a, b: a + b)

fd_list      = fd_counts.collect()
feat_dict    = dict(feat_marg.collect())
dens_dict    = dict(dens_marg.collect())

mi_relevance = {}
for (j, gval, dc), cnt in fd_list:
    p_joint = cnt / N_total
    p_g     = feat_dict[(j, gval)] / N_total
    p_d     = dens_dict[dc] / N_total
    mi_relevance[j] = mi_relevance.get(j, 0.0) + p_joint * math.log2(p_joint / (p_g * p_d))


#### Compute I(gi,gj) And Redundancy

In [8]:
pair_counts = (
    mr_rdd.flatMap(
        lambda row: [(((j, l, getattr(row, col_names[j]), getattr(row, col_names[l]))), 1)
                      for j in range(num_feats) for l in range(j + 1, num_feats)]
    ).reduceByKey(lambda a, b: a + b)
)

# aggregate pair dictionaries ( (j , l) , ( (v1,v2),cnt))
from collections import defaultdict
pair_dict = defaultdict(list)
for ((j, l, vj, vl), c) in pair_counts.collect():
    pair_dict[(j, l)].append(((vj, vl), c))

# compute mutual information
mi_pair = {}
for (j, l), items in pair_dict.items():
    score = 0.0
    for (vj, vl), cnt in items:
        p_joint = cnt / N_total
        p_j     = feat_dict[(j, vj)] / N_total
        p_l     = feat_dict[(l, vl)] / N_total
        score  += p_joint * math.log2(p_joint / (p_j * p_l))
    mi_pair[(j, l)] = score
    mi_pair[(l, j)] = score

#### Greedy mRMD Selection

In [9]:
# select the number of desired subspace
subspace_size = 10
selected, remaining = [], list(range(num_feats))

while remaining and len(selected) < subspace_size:
    best, best_score = None, float("-inf")
    for cand in remaining:
        redund = 0.0
        if selected:
            redund = sum(mi_pair.get((cand, s), 0.0) for s in selected) / len(selected)
        score = mi_relevance[cand] - redund
        if score > best_score:
            best, best_score = cand, score
    selected.append(best)
    remaining.remove(best)

# see the selected features
print("\nSelected features (MRMD order):", [f"g{i}" for i in selected])


Selected features (MRMD order): ['g44', 'g4', 'g146', 'g101', 'g156', 'g145', 'g93', 'g144', 'g66', 'g110']


#### Stage 5: Data Mapping

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark import StorageLevel

# Convert 'gXX' to integer indices and broadcast for tiny closures
indexes = [
    col if isinstance(col, int) else int(col[1:])   # 'g12' → 12, 12 → 12
    for col in selected
]
bc_idx = spark.sparkContext.broadcast(indexes)

# Add a row-id so we can join later if needed
norm_rdd = (
    df_scaled.rdd
            .zipWithIndex()
            .map(lambda t: (t[1], t[0].scaledFeatures))
)

proj_rdd = (
    norm_rdd
      .map(lambda kv: (kv[0],
                       [kv[1][i] for i in bc_idx.value]))
)


subspace_df = (
    proj_rdd
      .map(lambda kv: (kv[0], Vectors.dense(kv[1])))
      .toDF(["id", "subspaceFeatures"])
)

subspace_df.persist(StorageLevel.MEMORY_ONLY)

print(f"✓ Data-Mapping complete – projected into {len(indexes)}-D sub-space.")
subspace_df.show(5, truncate=False)


✓ Data-Mapping complete – projected into 10-D sub-space.


+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |subspaceFeatures                                                                                                                                                                                     |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |[0.061269146608315096,0.002257336343115124,0.06493506493506494,0.47506561679790027,0.03233830845771144,0.013623978201634877,0.03125,0.00510204081632653,0.0024937655860349127,0.02396514161220044]   |
|1  |[0.15098468271334792,0.002257336343115124,0.048701298701298704,0.4461942257217848,0.05472636815920398,0.010899182561307902,0.443359375,0.00510204081632653,0.0024937655860349127,0.

#### Stage 6: Compute LOF Scores
#### First Find K Nearest Neighbors

In [11]:
k = 50
rdd = subspace_df.rdd.map(lambda r: (r.id, r.subspaceFeatures))

pairs = (
    rdd.cartesian(rdd)
        .filter(lambda t: t[0][0] != t[1][0])
        .map(lambda t: ( t[0][0],
                         ( Vectors.squared_distance(t[0][1], t[1][1]),
                           t[1][0]) ))
)

def top_k(acc, x):
    if len(acc) < k:
        heapq.heappush(acc, (-x[0], x[1]))           # max-heap keeps k smallest
    else:
        heapq.heappushpop(acc, (-x[0], x[1]))
    return acc

knn = pairs.aggregateByKey([], top_k,
                           lambda a, b: heapq.nsmallest(k, a + b, key=lambda t: -t[0]))


#### Compute Local Reachability Density

In [12]:
kdist = knn.mapValues(lambda lst: max(-d2 for d2, _ in lst)).collectAsMap()
bc_kd  = spark.sparkContext.broadcast(kdist)

reach_rdd = knn.flatMap(
    lambda item: [
        (item[0], (max(-d2, bc_kd.value[j]), 1.0))
        for d2, j in item[1]
    ]
)

lrd = (reach_rdd
       .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
       .mapValues(lambda s: s[1] / s[0]))


#### LOF(i) = Σ LRD(j) / (k * LRD(i)

In [14]:
from pyspark.sql.types import StructType, StructField, LongType, DoubleType
from pyspark.storagelevel import StorageLevel

bc_lrd = spark.sparkContext.broadcast(lrd.collectAsMap())
k      = 50

schema = StructType([
    StructField("id",  LongType(),   False),
    StructField("lof", DoubleType(), False)
])

lof_rdd = knn.map(
    lambda item: (
        int(item[0]),
        float(
            sum(bc_lrd.value[j] for _, j in item[1]) /
            (k * bc_lrd.value[item[0]])
        )
    )
)


lof_df = (spark.createDataFrame(lof_rdd, schema)
                 .persist(StorageLevel.MEMORY_ONLY))

print("top-5 outliers (highest LOF):")
lof_df.orderBy("lof", ascending=False).show(5, truncate=False)
# lof_df.write.mode("overwrite").option("header", True).csv("output/lof_results")


top-5 outliers (highest LOF):


+---+------------------+
|id |lof               |
+---+------------------+
|807|18.935226716433124|
|243|17.948039433107876|
|241|17.946894738778937|
|242|17.946894738778937|
|796|17.90126260611107 |
+---+------------------+
only showing top 5 rows



In [15]:
lof_df.write.mode("overwrite").option("header", True).csv("output/lof_results")

#### Test With Different Number Of Workers

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd


pdf = pd.read_csv("musk2.csv")
csv_path = 'musk2.csv'

import subprocess, re

workers_list = [1, 2, 4, 8, 16,20,26,32,64]
results = []

for w in workers_list:
    print(f"→ Running with {w} worker{'s' if w>1 else ''}…")
    out = subprocess.check_output(
        f"python pipeline.py --csv {csv_path} --workers {w}",
        shell=True, stderr=subprocess.STDOUT
    ).decode()
    print(out)
    # parse Time=…s and AUC=…
    t_sec = float(re.search(r"Time=(\d+\.\d+)s", out).group(1))
    auc   = float(re.search(r"AUC=(\d+\.\d+)", out).group(1))
    results.append({"workers": w, "time_s": t_sec, "auc": auc})

#Build a DataFrame of results
results_df = pd.DataFrame(results)
results_df

→ Running with 1 worker…
25/07/07 04:10:54 WARN Utils: Your hostname, Soroush resolves to a loopback address: 127.0.1.1; using 192.168.100.10 instead (on interface wlp5s0)
25/07/07 04:10:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/07 04:10:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/07 04:10:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/07 04:10:59 WARN TaskSetManager: Stage 0 contains a task of very large size (4343 KiB). The maximum recommended task size is 1000 KiB.
25/07/07 04:11:00 WARN TaskSetManager: Stage 3 contains a task of very large size (4343 KiB). The maximum recommended task size is 1000 K

In [14]:
df.to_csv('results.csv', index=False)
df_copy = df

In [17]:
print("Spark version:", spark.version)

Spark version: 3.5.4
